# 1. 读取配置文件

In [ ]:
import json

In [ ]:
config_path = "../exps/momentum.json"
with open(config_path, 'r') as f:
    args = json.load(f)
print(args)

In [ ]:
import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# 2. 读取数据集

In [ ]:
import torchvision as tv
from torch.utils.data import DataLoader

In [ ]:
trsf = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean=(0.5071, 0.4867, 0.4408), std=(0.2675, 0.2565, 0.2761))
])
dataset = tv.datasets.CIFAR100(root='../data/CIFAR', train=False, transform=trsf, download=False)
dataloader = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=0)

In [ ]:
len(dataset)

# 2.1 tools

In [ ]:
import numpy as np

In [ ]:
def layernorm(_x, _eps=1e-05):
    """
    :param _x: ndarray of [n, dim]
    """
    _mean = np.mean(_x, axis=1, keepdims=True)
    _std = np.std(_x, axis=1, keepdims=True)
    _x = (_x - _mean) / (_std + _eps)
    return _x

In [ ]:
def extract_features(_extractor, _loader):
    _features = []
    _labels = []
    _extractor.to(device)
    _extractor.eval()
    for _i, (_x, _y) in tqdm(enumerate(_loader)):
        _x = _x.to(device)
        with torch.no_grad():
            _f = _extractor(_x)["features"]
        _features.append(_f.to("cpu"))
        _labels.append(_y)
    _features = torch.cat(_features, dim=0)
    _labels = torch.cat(_labels, dim=0)
    return _features, _labels

# 3. 读取模型

In [ ]:
# 训练时不使用norm
# saved_path = "../saved/noLN_momentum_resnet18_cifar100_50_all.pkl"
# 训练时使用norm
saved_path = "../saved/e250+50_momentum_resnet18_cifar100_50_all.pkl"

In [ ]:
import torch
from tqdm import tqdm
from utils.inc_net import HeadNet

In [ ]:
model = HeadNet(args, False, None)

In [ ]:
saved_all = torch.load(saved_path)
model.load_state_dict(saved_all["model_state_dict"], strict=False)
means = saved_all["means"]
stds = saved_all["stds"]

## 3.1 计算每个类的特征向量

In [ ]:
features, labels = extract_features(model.convnet, dataloader)
features = features.numpy()
labels = labels.numpy()

In [ ]:
print(features.shape, labels.shape)

# 4. 对特征进行t-SNE可视化

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
def select_classes(_classes):
    _features = []
    _labels = []
    for _i, _l in enumerate(labels):
        if _l in _classes:
            _features.append(features[_i])
            _labels.append(_l)
    _features = np.array(_features)
    _labels = np.array(_labels)
    return _features, _labels

In [ ]:
def show_tsne(_classes, _features, _labels, _colors):
    _tsne = TSNE(n_components=2, init='pca', random_state=0)
    _features = _tsne.fit_transform(_features)  # [n, 2]
    _class_features = [_features[_labels == i] for i in _classes]

    for i in range(len(_classes)):
        plt.scatter(_class_features[i][:, 0], _class_features[i][:, 1], c=_colors[i], label=_classes[i])
    plt.axis('off')
    plt.legend()
    plt.show()

## 4.1 旧类

In [ ]:
selected_classes = [0, 15, 30, 45]
colors = ["#000000", "#BEBEBE", "#AE0000", "#8600FF"]
selected_features, selected_labels = select_classes(selected_classes)

In [ ]:
# 不使用norm
show_tsne(selected_classes, selected_features, selected_labels, colors)

In [ ]:
# 对特征向量使用norm
show_tsne(selected_classes, layernorm(selected_features), selected_labels, colors)

## 4.2 新类

In [ ]:
selected_classes = [50, 65, 80, 95]
colors = ["#2828FF", "#00FFFF", "#00EC00", "#F9F900"]
selected_features, selected_labels = select_classes(selected_classes)

In [ ]:
# 不使用norm
show_tsne(selected_classes, selected_features, selected_labels, colors)

In [ ]:
# 对特征向量使用norm
show_tsne(selected_classes, layernorm(selected_features), selected_labels, colors)